In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.layers import LSTM 
from keras.layers import Embedding, Dense, Dropout
from keras.layers import LeakyReLU

Using TensorFlow backend.


# Function to perform pre-processing steps

At first tokenization will be performed then stemming will be performed over tokens

In [2]:
def token_stems(text):
        
    tokens=tokenizing(text) 
    stems=stemming(tokens)
    return stems       

# Stemming Function

In [3]:
from nltk.stem.snowball import SnowballStemmer
stemmer= SnowballStemmer("english")


def stemming(text):
    
    stems =[stemmer.stem(t) for t in text]
    return stems

# Tokenization Function

In [4]:
def tokenizing(text):
    
    #breaking each word and making them tokens
    tokens=[word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    #storing only alpha tokens
    filtered_tokens=[]
    for token in tokens:
        if (re.search('[a-zA-Z]|\'', token)):
            filtered_tokens.append(token)

    return filtered_tokens

In [5]:
train= pd.read_excel("D:\\virtual\\dataset.xlsx")
#train= train[train['Module']== 'Personal']
train.shape

(1565, 7)

In [6]:
docs= train['Data']

In [7]:
train.head()

,S.no,Data,Module,Type,sub_type,sub_type_two,sub_type_three
0,1,What is my department?,Personal,Department,particular,empty,department
1,2,What is my department name?,Personal,Department,particular,empty,department
2,3,In which department I belong?,Personal,Department,particular,empty,department
3,4,In which department I am working in?,Personal,Department,particular,empty,department
4,5,What is the name of department?,Personal,Department,particular,empty,department


In [8]:
tokens = []
for i in docs:
    temp = token_stems(i)
    tokens.append(temp)


In [9]:
#print(docs[1],"\n\n",tokens[1])

In [10]:
x, y = np.asarray(tokens) , np.asarray(train['sub_type'])

In [11]:
le = LabelEncoder()
y = le.fit_transform(y)
y= to_categorical(y)
#y[:10]

In [12]:
xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size= 0.3, random_state=100)

In [13]:
max_words = 20000
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(x)

In [14]:
sequences = tok.texts_to_sequences(x)
test_sequences = tok.texts_to_sequences(xtest)

In [15]:
max_len =200

sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)
#test_sequences_matrix[1]

# Model 

In [16]:
def model():
    
    #Create model by simply calling the Sequential constructor
    
    model = Sequential()
    
    #This layer can only be used as the first layer in a model.
    
    model.add(Embedding(max_words,50,input_length=max_len))
    
    #specify number of neurons in lstm layer and the activation function that we want to use.
    
    model.add(LSTM(64, activation='tanh',dropout=0.01))
    
    #specify number of neurons in dense layer and the activation function that we want to use.
    
    model.add(Dense(50, activation='relu'))
    
    #add dropout to prevent overfitting
    
    model.add(Dropout(0.01))
    
    #and we create our output layer with two nodes as we have 2 class labels.
    
    model.add(Dense(2, activation='softmax'))
    
    #Now for training, we need to define an optimizer, loss measure and the error metric.
    # We will use the binary_crossentropy as our loss measure. 
    #As for the minimization algorithm, we will use "rmsprop".
    #This optimizer is usually a good choice for recurrent neural networks.
    
    
    model.compile(loss='binary_crossentropy', optimizer='rmsprop',metrics=['accuracy'])
    return model

In [17]:
c= model()

c.fit(sequences_matrix,y,epochs=24, validation_split=0.3)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 1095 samples, validate on 470 samples
Epoch 1/24
1095/1095 [==============================] - 6s 5ms/step - loss: 0.3998 - acc: 0.8365 - val_loss: 0.4676 - val_acc: 0.8000
Epoch 2/24
1095/1095 [==============================] - 5s 5ms/step - loss: 0.0775 - acc: 0.9826 - val_loss: 0.6264 - val_acc: 0.7979
Epoch 3/24
1095/1095 [==============================] - 5s 5ms/step - loss: 0.0428 - acc: 0.9854 - val_loss: 0.7518 - val_acc: 0.8085
Epoch 4/24
1095/1095 [==============================] - 5s 5ms/step - loss: 0.0176 - acc: 0.9954 - val_loss: 1.0094 - val_acc: 0.8043
Epoch 5/24
1095/1095 [==============================] - 6s 5ms/step - loss: 0.0136 - acc: 0.9945 - val_loss: 1.1023 - val_acc: 0.8064
Epoch 6/24
1095/1095 [==========================

In [18]:
"""from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit


m = model()
# evaluate model with standardized dataset
estimator = KerasClassifier(build_fn=model, epochs=15, verbose=2)
kfold = ShuffleSplit(n_splits=15, test_size=0.3, random_state=100)
m.fit(cross_val_score(estimator,sequences_matrix,y, cv=kfold))"""


'from keras.layers import Dense\nfrom keras.models import Sequential\nfrom keras.wrappers.scikit_learn import KerasClassifier\nfrom sklearn.model_selection import StratifiedKFold\nfrom sklearn.model_selection import cross_val_score\nfrom sklearn.model_selection import ShuffleSplit\n\n\nm = model()\n# evaluate model with standardized dataset\nestimator = KerasClassifier(build_fn=model, epochs=15, verbose=2)\nkfold = ShuffleSplit(n_splits=15, test_size=0.3, random_state=100)\nm.fit(cross_val_score(estimator,sequences_matrix,y, cv=kfold))'

In [19]:
accuracy = c.evaluate(test_sequences_matrix,ytest)

470/470 [==============================] - 0s 755us/step


In [21]:
#save your model
c.save('Leave.h5')